# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

K. El-Badry  ->  K. El-Badry  |  ['K. El-Badry']
E. Schinnerer  ->  E. Schinnerer  |  ['E. Schinnerer']
C. Desgrange  ->  C. Desgrange  |  ['C. Desgrange']
J. Olofsson  ->  J. Olofsson  |  ['J. Olofsson']
F. Walter  ->  F. Walter  |  ['F. Walter']
L. Boogaard  ->  L. Boogaard  |  ['L. Boogaard']


Arxiv has 66 new papers today
          5 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/5 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2309.15143


extracting tarball to tmp_2309.15143...

 done.
Retrieving document from  https://arxiv.org/e-print/2309.15150


extracting tarball to tmp_2309.15150...

...

 done.
Retrieving document from  https://arxiv.org/e-print/2309.15671


extracting tarball to tmp_2309.15671...

 done.


F. Walter  ->  F. Walter  |  ['F. Walter']
L. Boogaard  ->  L. Boogaard  |  ['L. Boogaard']


Found 170 bibliographic references in tmp_2309.15671/Manuscript.bbl.
syntax error in line 11: '=' expected


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2309.15150-b31b1b.svg)](https://arxiv.org/abs/arXiv:2309.15150) | **A$^3$COSMOS: the infrared luminosity function and dust-obscured star  formation rate density at $0.5<z<6$**  |
|| A. Traina, et al. -- incl., <mark>E. Schinnerer</mark> |
|*Appeared on*| *2023-09-28*|
|*Comments*| *17 pages, 15 figures, 5 tables, accepted for publication on A&A*|
|**Abstract**| Aims: We leverage the largest available Atacama Large Millimetre/submillimetre Array (ALMA) survey from the archive (A$^3$COSMOS) to study to study infrared luminosity function and dust-obscured star formation rate density of sub-millimeter/millimeter (sub-mm/mm) galaxies from $z=0.5\,-\,6$. Methods: The A$^3$COSMOS survey utilizes all publicly available ALMA data in the COSMOS field, therefore having inhomogeneous coverage in terms of observing wavelength and depth. In order to derive the luminosity functions and star formation rate densities, we apply a newly developed method that corrects the statistics of an inhomogeously sampled survey of individual pointings to those representing an unbiased blind survey. Results: We find our sample to mostly consist of massive ($M_{\star} \sim 10^{10} - 10^{12}$ $\rm M_{\odot}$), IR-bright ($L_* \sim 10^{11}-10^{13.5} \rm L_{\odot}$), highly star-forming (SFR $\sim 100-1000$ $\rm M_{\odot}$ $\rm yr^{-1}$) galaxies. We find an evolutionary trend in the typical density ($\Phi^*$) and luminosity ($L^*$) of the galaxy population, which decrease and increase with redshift, respectively. Our IR LF is in agreement with previous literature results and we are able to extend to high redshift ($z > 3$) the constraints on the knee and bright-end of the LF, derived by using the Herschel data. Finally, we obtain the SFRD up to $z\sim 6$ by integrating the IR LF, finding a broad peak from $z \sim 1$ to $z \sim 3$ and a decline towards higher redshifts, in agreement with recent IR/mm-based studies, within the uncertainties, thus implying the presence of larger quantities of dust than what is expected by optical/UV studies. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2309.15158-b31b1b.svg)](https://arxiv.org/abs/arXiv:2309.15158) | **The first scattered light images of HD 112810, a faint debris disk in  the Sco-Cen association**  |
|| E. C. Matthews, et al. -- incl., <mark>C. Desgrange</mark>, <mark>J. Olofsson</mark> |
|*Appeared on*| *2023-09-28*|
|*Comments*| *A&A accepted. 13 pages, 6 figures + appendix*|
|**Abstract**| Context: Circumstellar debris disks provide insight into the formation and early evolution of planetary systems. Resolved belts in particular help to locate planetesimals in exosystems, and can hint at the presence of disk-sculpting exoplanets. Aims: We study the circumstellar environment of HD 112810 (HIP 63439), a mid-F type star in the Sco-Cen association with a significant infrared excess indicating the presence of a circumstellar debris disk. Methods: We collected five high-contrast observations of HD 112810 with VLT/SPHERE. We identified a debris disk in scattered light, and found that the debris signature is robust over a number of epochs and a variety of reduction techniques. We modelled the disk, accounting for self-subtraction and assuming that it is optically thin. Results: We find a single-belt debris disk, with a radius of 118$\pm$9au and an inclination angle of ${75.7}^{+1.1}_{-1.3}$$\deg$. This is in good agreement with the constraints from SED modelling and from a partially-resolved ALMA image of the system. No planets are detected, though planets below the detection limit ($\sim$2.6M$_\textrm{J}$ at a projected separation of 118au) could be present and could have contributed to sculpting the ring of debris. Conclusions: HD 112810 adds to the growing inventory of debris disks imaged in scattered light. The disk is faint, but the radius and the inclination of the disk are promising for follow-up studies of the dust properties. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2309.15671-b31b1b.svg)](https://arxiv.org/abs/arXiv:2309.15671) | **MIDIS: Unveiling the Role of Strong Ha-emitters during the Epoch of  Reionization with JWST**  |
|| P. Rinaldi, et al. -- incl., <mark>F. Walter</mark>, <mark>L. Boogaard</mark> |
|*Appeared on*| *2023-09-28*|
|*Comments*| *21 pages, 12 Figures, 1 table. Submitted in ApJ. Comments are welcome*|
|**Abstract**| We make use of the deepest JWST/MIRI image at 5.6 um, obtained in the Hubble eXtreme Deep Field (XDF), to constrain the role of strong Ha emitters (HAEs) in Cosmic Reionization at z~7-8. Our sample of bright (M(UV) < -20 mag) HAEs is comprised of young (<30 Myr) galaxies with low stellar masses (<= 10^9 Msun). They span a wide range of UV-beta slopes, with a median beta = -2.22+-0.35, which broadly correlates with stellar mass. We estimate the ionizing photon production efficiency (xi_ion,0) of these sources (assuming f_esc,LyC = 0), which yields a median value log10(xi_ion,0/(Hz erg^(-1))) = 25.54(+0.09, -0.10). We show that xi_ion,0 positively correlates with EW0(Ha) and specific star formation rate (sSFR). Instead xi_ion,0 weakly anti-correlates with stellar mass and beta. Based on the beta values, we estimate f_esc,LyC=0.07(+0.03, -0.02), which results in log10(xi_ion/(Hz erg^(-1))) = 25.59 (+0.06, -0.04). By considering this result along with others from the literature, we find a mild evolution of xi_ion with redshift. Finally, we assess the impact of strong HAEs during Cosmic Reionization at z~7-8. We find that our HAEs do not need high values of f_esc, rel (only 6-10%) to be able to reionize their surrounding intergalactic medium. They have N_dot_ion = 10^(50.43+-0.3) s^(-1)Mpc^(-3) and contribute more than a factor of two in terms of emitted ionizing photons per comoving volume compared to non-Ha emitters in the same redshift bin, suggesting that strong, young, and low stellar-mass emitters could have played a central role during the Epoch of Reionization. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2309.15143-b31b1b.svg)](https://arxiv.org/abs/arXiv:2309.15143) | **Triage of the Gaia DR3 astrometric orbits. II. A census of white dwarfs**  |
|| S. Shahaf, et al. -- incl., <mark>K. El-Badry</mark> |
|*Appeared on*| *2023-09-28*|
|*Comments*| *Submitted to MNRAS*|
|**Abstract**| The third data release of Gaia was the first to include orbital solutions assuming non-single stars. Here, we apply the astrometric triage technique of Shahaf et al. 2019 to identify binary star systems with companions that are not single main-sequence stars. Gaia's synthetic photometry of these binaries is used to distinguish between systems likely to have white-dwarf companions and those that may be hierarchical triples. The study uncovered a population of nearly 3200 binaries, characterised by orbital separations on the order of an astronomical unit, in which the faint astrometric companion is probably a white dwarf. Remarkably, over 110 of these systems exhibit significant ultraviolet excess flux, confirming this classification and, in some cases, indicating their relatively young cooling ages. We show that the sample is not easily reproduced by binary population synthesis codes. Therefore, it challenges current binary evolution models, offering a unique opportunity to gain insights into the processes governing white-dwarf formation, binary evolution, and mass transfer. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2309.15310-b31b1b.svg)](https://arxiv.org/abs/arXiv:2309.15310) | **Microlensing Discovery and Characterization Efficiency in the Vera C.  Rubin Legacy Survey of Space and Time**  |
|| N. S. Abrams, et al. |
|*Appeared on*| *2023-09-28*|
|*Comments*| *21 pages, 15 figures, submitted to ApJS Rubin Survey Strategy Edition*|
|**Abstract**| The Vera C. Rubin Legacy Survey of Space and Time will discover thousands of microlensing events across the Milky Way Galaxy, allowing for the study of populations of exoplanets, stars, and compact objects. It will reach deeper limiting magnitudes over a wider area than any previous survey. We evaluate numerous survey strategies simulated in the Rubin Operation Simulations (OpSims) to assess the discovery and characterization efficiencies of microlensing events. We have implemented three metrics in the Rubin Metric Analysis Framework: a discovery metric and two characterization metrics, where one estimates how well the lightcurve is covered and the other quantifies how precisely event parameters can be determined. We also assess the characterizability of microlensing parallax, critical for detection of free-floating black hole lenses, in a representative bulge and disk field. We find that, given Rubin's baseline cadence, the discovery and characterization efficiency will be higher for longer duration and larger parallax events. Microlensing discovery efficiency is dominated by observing footprint, where more time spent looking at regions of high stellar density including the Galactic bulge, Galactic plane, and Magellanic clouds, leads to higher discovery and characterization rates. However, if the observations are stretched over too wide an area, including low-priority areas of the Galactic plane with fewer stars and higher extinction, event characterization suffers by > 10%, which could impact exoplanet, binary star, and compact object events alike. We find that some rolling strategies (where Rubin focuses on a fraction of the sky in alternating years) in the Galactic bulge can lead to a 15-20% decrease in microlensing parallax characterization, so rolling strategies should be chosen carefully to minimize losses. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Planck' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2309.15150.md
    + _build/html/tmp_2309.15150/./areal_coverage/areal_coverage.png
    + _build/html/tmp_2309.15150/./LF/lstar_phistar.png
    + _build/html/tmp_2309.15150/./LF/lf_a3only.png
exported in  _build/html/2309.15158.md
    + _build/html/tmp_2309.15158/./figures/irdis_grid.png
    + _build/html/tmp_2309.15158/./figures/data_model_residuals/grater_model_5par.png
    + _build/html/tmp_2309.15158/./figures/data_model_residuals/grater_model_6par.png
    + _build/html/tmp_2309.15158/./figures/data_model_residuals/grater_model_7par.png
    + _build/html/tmp_2309.15158/./figures/hip63439_planetdetectionlim.png
exported in  _build/html/2309.15671.md
    + _build/html/tmp_2309.15671/./beta_vs_age_models.png
    + _build/html/tmp_2309.15671/./Beta_vs_Mass.png
    + _build/html/tmp_2309.15671/./Xion_f_esc_MUV.png


## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\arraystretch}{1.5}$
$\newcommand{\arraystretch}{1.5}$
$\newcommand{\arraystretch}{1.5}$
$\newcommand{\arraystretch}{1.5}$
$\newcommand{\arraystretch}{1.5}$
$\newcommand{\cha}{\textit{Chandra}}$
$\newcommand{\XMM}{{XMM-\textit{Newton}}}$
$\newcommand{\NuSTAR}{\textit{NuSTAR}}$
$\newcommand{\Nu}{\textit{NuSTAR,}}$
$\newcommand{\bat}{{{\it Swift}-BAT}}$
$\newcommand{\}{XSPEC}$
$\newcommand{\}{pexrav}$
$\newcommand{\}{MYTorus}$
$\newcommand{\}{borus}$
$\newcommand{\}{bntorus}$
$\newcommand{\}{ngc}$
$\newcommand{\}{a}$
$\newcommand{\}{ad}$</div>



<div id="title">

# A$^3$COSMOS: the infrared luminosity function and dust-obscured star formation rate density at $0.5<z<6$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2309.15150-b31b1b.svg)](https://arxiv.org/abs/2309.15150)<mark>Appeared on: 2023-09-28</mark> -  _17 pages, 15 figures, 5 tables, accepted for publication on A&A_

</div>
<div id="authors">

A. Traina, et al. -- incl., <mark>E. Schinnerer</mark>

</div>
<div id="abstract">

**Abstract:** Aims: We leverage the largest available Atacama Large Millimetre/submillimetre Array (ALMA) survey from the archive (A$^3$COSMOS) to study to study infrared luminosity function and dust-obscured star formation rate density of sub-millimeter/millimeter (sub-mm/mm) galaxies from $z=0.5\,-\,6$. Methods: The A$^3$COSMOS survey utilizes all publicly available ALMA data in the COSMOS field, therefore having inhomogeneous coverage in terms of observing wavelength and depth. In order to derive the luminosity functions and star formation rate densities, we apply a newly developed method that corrects the statistics of an inhomogeously sampled survey of individual pointings to those representing an unbiased blind survey. Results: We find our sample to mostly consist of massive ($M_{\star} \sim 10^{10} - 10^{12}$ $\rm M_{\odot}$), IR-bright ($L_* \sim 10^{11}-10^{13.5} \rm L_{\odot}$), highly star-forming (SFR $\sim 100-1000$ $\rm M_{\odot}$ $\rm yr^{-1}$) galaxies. We find an evolutionary trend in the typical density ($\Phi^*$) and luminosity ($L^*$) of the galaxy population, which decrease and increase with redshift, respectively. Our IR LF is in agreement with previous literature results and we are able to extend to high redshift ($z > 3$) the constraints on the knee and bright-end of the LF, derived by using the Herschel data. Finally, we obtain the SFRD up to $z\sim 6$ by integrating the IR LF, finding a broad peak from $z \sim 1$ to $z \sim 3$ and a decline towards higher redshifts, in agreement with recent IR/mm-based studies, within the uncertainties, thus implying the presence of larger quantities of dust than what is expected by optical/UV studies. 

</div>

<div id="div_fig1">

<img src="tmp_2309.15150/./areal_coverage/areal_coverage.png" alt="Fig6" width="100%"/>

**Figure 6. -** Total areal coverage of the 1155 pointings after the cut for the lack of target detection within 1 arcsec.\\ (*fig:areal_coverage*)

</div>
<div id="div_fig2">

<img src="tmp_2309.15150/./LF/lstar_phistar.png" alt="Fig8" width="100%"/>

**Figure 8. -** Evolutionary trends of $\Phi^*$ (top panel) and $L^*$ (bottom panel) with redshift. The solid dark-red lines and shaded areas show the behaviours of $\Phi^*$ and $L^*$ at all the redshifts, while pink diamonds are the estimates obtained fitting each redshift bin individually in the ALMA only case. Finally, black empty circles and yellow points represent estimates for $L^*$ and $\Phi^*$ from \cite{gruppioni2013lf and \cite{gruppioni2020alpine}, reported for comparison.} (*fig:lstar_phistar*)

</div>
<div id="div_fig3">

<img src="tmp_2309.15150/./LF/lf_a3only.png" alt="Fig13" width="100%"/>

**Figure 13. -** A$^3$COSMOS luminosity function (black points and red boxes). The individual redshift bin MCMC best-fit is plotted as pink solid lines and shaded error bands. The redshift ranges are reported in the upper-right corner of each subplot, while the luminosity bins are centered at each 0.25 dex, with a width of 0.5 dex (overlapping bins). The black vertical dashed lines represent the completeness limit of the L$_{IR$. The orange dashed, purple dotted and green dashed lines are the best fit LF obtained by \cite{gruppioni2013lf,magnelli2013ir,gruppioni2020alpine}.} (*fig:LF_models_a3_only*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2309.15150"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\diskpa}{{98.6\pm0.7}\deg}$
$\newcommand{\diskincl}{{75.7}^{+1.1}_{-1.3}\deg}$
$\newcommand{\diskrad}{{118\pm9}\textrm{au}}$
$\newcommand{\deg}{^\circ}$
$\newcommand{\arraystretch}{1.5}$
$\newcommand{\arraystretch}{0.666}$
$\newcommand{\arraystretch}{1.3}$
$\newcommand{\arraystretch}{0.769}$
$\newcommand{\refereeresponse}{\latexdiff}$
$\newcommand{\Lir}{L_{\textrm{IR}}/L_{\ast}}$
$\newcommand{\gaia}{\textit{Gaia}}$
$\newcommand{\hipp}{\textit{Hipparcos}}$
$\newcommand{\apjl}{ApJL}$
$\newcommand{\aj}{AJ}$
$\newcommand{\apj}{ApJ}$
$\newcommand{\apjs}{ApJS}$
$\newcommand{\pasp}{PASP}$
$\newcommand{\pasj}{PASJ}$
$\newcommand{\spie}{SPIE}$
$\newcommand{\apjs}{ApJS}$
$\newcommand{\araa}{ARAA}$
$\newcommand{\aap}{A\&A}$
$\newcommand{\aaps}{A\&AS}$
$\newcommand{\apss}{Ap\&SS}$
$\newcommand{\mnras}{MNRAS}$
$\newcommand{\memsai}{MmSAI}$
$\newcommand{\sovast}{SvA}$
$\newcommand{\rmxaa}{RMxAA}$
$\newcommand{\nat}{Nature}$
$\newcommand{\sci}{Science}$
$\newcommand{\etal}{{et al.}}$</div>



<div id="title">

# The first scattered light images of HD 112810, a faint debris disk in the Sco-Cen association

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2309.15158-b31b1b.svg)](https://arxiv.org/abs/2309.15158)<mark>Appeared on: 2023-09-28</mark> -  _A&A accepted. 13 pages, 6 figures + appendix_

</div>
<div id="authors">

E. C. Matthews, et al. -- incl., <mark>C. Desgrange</mark>, <mark>J. Olofsson</mark>

</div>
<div id="abstract">

**Abstract:** Circumstellar debris disks provide insight into the formation and early evolution of planetary systems. Resolved belts in particular help to locate planetesimals in exosystems, and can hint at the presence of disk-sculpting exoplanets. We study the circumstellar environment of $\object{HD 112810}$ (HIP 63439), a mid-F type star in the Sco-Cen association with a significant infrared excess indicating the presence of a circumstellar debris disk. We collected five high-contrast observations of HD 112810 with VLT/SPHERE. We identified a debris disk in scattered light, and found that the debris signature is robust over a number of epochs and a variety of reduction techniques. We modelled the disk, accounting for self-subtraction and assuming that it is optically thin. We find a single-belt debris disk, with a radius of $\diskrad$ and an inclination angle of $\diskincl$ . This is in good agreement with the constraints from SED modelling and from a partially-resolved ALMA image of the system. No planets are detected, though planets below the detection limit ( $\sim$ 2.6M $_\textrm{J}$ at a projected separation of 118au) could be present and could have contributed to sculpting the ring of debris. HD 112810 adds to the growing inventory of debris disks imaged in scattered light. The disk is faint, but the radius and the inclination of the disk are promising for follow-up studies of the dust properties.

</div>

<div id="div_fig1">

<img src="tmp_2309.15158/./figures/irdis_grid.png" alt="Fig1" width="100%"/>

**Figure 1. -** SPHERE/IRDIS images of the HD 112810 debris disk. Each column indicates data processed via a different method: GRAPHIC/PCA \citep[Sect. \ref{sec:dr-graphic};][]{hagelberg2016}, TLOCI as provided by the SPHERE Data Center \citep[Sect. \ref{sec:spheredc};][]{delorme2017} or RDI \citep[Sect. \ref{sec:rdi};][]{xie2022}; rows indicate each observation epoch. In each case, the two SPHERE/IRDIS channels are co-added. There are no RDI reductions for the BB\_H data, since there is far more data collected with the DB\_H23 filter in the archive, and the reference library we considered does not collate BB\_H data  ([Xie, Choquet and Vigan 2022]()) . The `normalised signal' is calculated relative to the background noise in the wide-field; note that this is _not_ a true SNR at very small separations, where the noise is higher than the background limit. (*fig:sphere_irdis_images*)

</div>
<div id="div_fig2">

<img src="tmp_2309.15158/./figures/data_model_residuals/grater_model_5par.png" alt="Fig3.1" width="33%"/><img src="tmp_2309.15158/./figures/data_model_residuals/grater_model_6par.png" alt="Fig3.2" width="33%"/><img src="tmp_2309.15158/./figures/data_model_residuals/grater_model_7par.png" alt="Fig3.3" width="33%"/>

**Figure 3. -** Best fitting models to the data, alongside the raw data and the residual image with the disk subtracted. 3- and 5-$\sigma$ contours (relative to the background noise) are shown in pink and purple respectively. The three rows show the best fitting models for (a) the 5-parameter fit (b) the vertical structure fit, and (c) the radial structure fit (see Sects. \ref{sec:5parfit}--\ref{sec:diskradial}). Best-fit parameters are given in Table \ref{tab:disk_parameters}. Residual images are generated by subtracting model disks from the raw data, and then repeating the PCA reduction. The debris disk is well-subtracted in all cases. (*fig:gratermodel*)

</div>
<div id="div_fig3">

<img src="tmp_2309.15158/./figures/hip63439_planetdetectionlim.png" alt="Fig4" width="100%"/>

**Figure 4. -** Contrast (left) and mass sensitivities (right) of each epoch of observations. The legend applies to both figures; cool colours indicate the narrowband DB\_H23 observations and warm colours indicate the broadband BB\_H observations. Companion magnitude limits were converted to mass limits using the ATMO models from [Phillips, Tremblin and Baraffe (2020)](). (*fig:contrastcurves*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2309.15158"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\gsim}{{\;\raise0.3ex\hbox{>\kern-0.75em\raise-1.1ex\hbox{\sim}}\;}}$</div>



<div id="title">

# $\bf{MIDIS:  Unveiling the Role of Strong H$\alpha$-emitters during the Epoch of Reionization with _JWST_}$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2309.15671-b31b1b.svg)](https://arxiv.org/abs/2309.15671)<mark>Appeared on: 2023-09-28</mark> -  _21 pages, 12 Figures, 1 table. Submitted in ApJ. Comments are welcome_

</div>
<div id="authors">

P. Rinaldi, et al. -- incl., <mark>F. Walter</mark>, <mark>L. Boogaard</mark>

</div>
<div id="abstract">

**Abstract:** We make use of the deepest _JWST_ /MIRI image at 5.6 $\mu m$ , in the Hubble eXtreme Deep Field, to constrain the role of strong H $\alpha$ emitters (HAEs) in Cosmic Reionization at $z\simeq7-8$ . Our sample of bright (M $_{UV} \lesssim -20$ mag) HAEs is comprised of young ( $<30\;\rm Myr$ ) galaxies with low stellar masses ( $\lesssim 10^{9}\;\rm M_{\odot}$ ). They span a wide range of UV- $\beta$ slopes, with a median $\beta = -2.22\pm0.35$ , which broadly correlates with stellar mass. We estimate the ionizing photon production efficiency ( $\xi_{ion,0}$ ) of these sources (assuming $f_{esc,LyC} = 0$ ), which yields a median value $\rm log_{10}(\xi_{ion,0}/(Hz\;erg^{-1})) = 25.54^{+0.09}_{-0.10}$ . We show that $\xi_{ion,0}$ positively correlates with EW $_{0}$ (H $\alpha$ ) and specific star formation rate (sSFR). Instead $\xi_{ion,0}$ weakly anti-correlates with stellar mass and $\beta$ . Based on the $\beta$ values, we estimate $f_{esc, LyC}=0.07^{+0.03}_{-0.02}$ , which results in $\rm log_{10}(\xi_{ion}/(Hz\;erg^{-1})) = 25.59^{+0.06}_{-0.04}$ . By considering this result along with others from the literature, we find a mild evolution of $\xi_{ion}$ with redshift. Finally, we assess the impact of strong HAEs during Cosmic Reionization at $z\simeq7-8$ . We find that our HAEs do not need high values of $f_{esc, rel}$ (only $6-10\%$ ) to be able to reionize their surrounding intergalactic medium. They have $\dot N_{ion} = 10^{50.43\pm0.3}\;\rm s^{-1}Mpc^{-3}$ and contribute more than a factor of two in terms of emitted ionizing photons per comoving volume compared to non-H $\alpha$ emitters in the same redshift bin, suggesting that strong, young, and low stellar-mass emitters could have played a central role during the Epoch of Reionization.

</div>

<div id="div_fig1">

<img src="tmp_2309.15671/./beta_vs_age_models.png" alt="Fig9" width="100%"/>

**Figure 9. -** Beta slope as a function of galaxy age. The ages of galaxies have been obtained as output from LePHARE. The red dashed line refers to the median $\beta$ value that we find in our sample, which is in line with what we expect from galaxies at high redshifts. For comparison, we also include theoretical predictions by considering synthetic-model tracks from \citet{Schaerer_2002, Schaerer_2003}, which are color-coded based on metallicity. Solid lines refer to models with a combination of stellar and nebular contributions, while dashed lines refer to pure stellar models. Two different SFHs have been adopted: burst and constant star formation. (*beta_vs_age_models*)

</div>
<div id="div_fig2">

<img src="tmp_2309.15671/./Beta_vs_Mass.png" alt="Fig2" width="100%"/>

**Figure 2. -** UV-$\beta$ slope as a function of stellar mass. A collection of results at high redshift from the recent literature is presented as well \citep{Wilkins_2016, Bhatawdekar_2021, Endsley_2021, Tacchella_2022, Topping_2022, Austin_2023, Bouwens_2023, Franco_2023, Mascia_2023}. From this plot, we can see that our sample of HAEs is dominated by low-mass galaxies ($\rm M_{\star} \leq 10^{9}\; M_{\odot}$). We also show colored regions (blue gradients) that correspond to the averages of the escape fraction of the Lyman continuum photons (5, 10, 20 per cent) by adopting Equation 11 from \citet{Chisholm_2022}. We include the $z \simeq 7$ relation from \cite{Finkelstein_2012b} as the dashed line. The purple shaded area refers to Delphi simulations at $z\simeq7$,  where we show how the nebular contribution (both continuum and emission lines) can impact the UV-$\beta$ slope as a function of $\rm M_{\star}$. Particularly, the lower limit of the shaded area refers to the pure stellar continuum + dust. The upper limit, instead, refers to the maximum contribution of stellar + nebular continuum + nebular lines + dust. (*Beta_vs_Mass*)

</div>
<div id="div_fig3">

<img src="tmp_2309.15671/./Xion_f_esc_MUV.png" alt="Fig3" width="100%"/>

**Figure 3. -** $\xi_{ion,0}$ as a function of M$_{UV}$.
    We also collect data points from the recent literature at high redshifts \citep{Harikane_2018, Lam_2019, Maseda_2020, Castellano_2022, Endsley_2022, Matthee_2022, Bunker_2023, Fujimoto_2023, Hsiao_2023, Lin_2023, Ning_2023, Prieto_Lyon_2023, Tang_2023}. Our sample at $z\simeq 7-8$ is comprised of relatively bright galaxies with M$_{UV} \lesssim -20$ mag (shaded area), given their redshifts and the depth of our analysed data. The red dot-dashed line indicates the median $\xi_{ion,0}$ value that we find in our sample. The black dashed line refers to the relation between $\xi_{ion,0}$ and M$_{UV}$, as reported in \citet{Prieto_Lyon_2023}. The green shaded area refers to the canonical value of $\xi_{ion}$ that is usually assumed \citep{Robertson_2013}. No evident correlation arises by comparing these two quantities. (*Xion_f_esc_MUV*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2309.15671"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

282  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

6  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

4  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6

  6 publications selected.
